# Parte 2 - HT Settings 

Nesta etapa, já temos a pergunta e o objeto de estudo oriundas da Parte 1: CaAlF5 (mp-8836)

Vamos estabelecer que a similaridade química será definida por elementos do mesmo grupo químico. Vamos desconsiderar o Rádio (Ra) e o Ástato (At).

Como *solver* de Mecânica Quântica, vamos utilizar o software VASP.

Estamos assumindo que as contas serão realizadas em um cluster com gerenciador de fila (como o SLURM) e que não iremos utilizar nenhuma biblioteca adicional para auxiliar no gerenciamento das contas, exceto aquelas de pré-processamento (Parte 2) e pós-processamento (Parte 3).

Em caso de uso de clusters sem fila, sugere-se uso da biblioteca ASE (https://wiki.fysik.dtu.dk/ase/) para submissão de cálculos direto pelo código Python. Para usuários mais avançados que requeiram altas vazões de contas, inclusive em clusters com gereciamento de fila, sugere-se o uso do AiiDA (https://www.aiida.net/).

-------------------

### Bibliotecas

In [ ]:
from pymatgen.io import vasp                                       # Interface with VASP
from pymatgen.core.structure import Structure                      # Class to represent Structures in pymatgen
from pymatgen.core.composition import Composition                  # Class to represent Composition in pymatgen
from pymatgen.core.periodic_table import Element                   # Class to represent Element in pymatgen
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer          # Methods to analyze space groups of materials

import shutil
import os

### Criando novos materiais e configurando arquivos VASP

Nessa primeira etapa estamos interessados em fazer a atualização das posições ionicas para encontrar as estruturas com menor enegia (convergidas).

Vamos utilizar a estrutura original do CaAlF5 como protótipo e manter a simetria, trocando apenas os átomos dos elementos da mesma coluna da tabela periódica.

Primeiro vamos carregar nosso protótipo:

In [ ]:
input_folder = 'ht/proto_CaAlF5'  # Pasta onde está a estrutura inicial (POSCAR) usada como protótipo
proto_poscar = Structure.from_file(input_folder+"/POSCAR")  # Estrutura protótipo

In [ ]:
proto_poscar

In [ ]:
proto_space_group = SpacegroupAnalyzer(proto_poscar, symprec=0.1, angle_tolerance=1.0)
proto_space_group.get_space_group_number()

Vamos usar as seguintes configurações de arquivos VASP:

In [ ]:
incar_rx = {
"ALGO": "Normal",
"EDIFF": "0.000001",
"ENCUT": 520,
"IBRION": 2,
"ISIF": 3,
"ISMEAR": 0,
"ISPIN": 1,
"LASPH": True,
"LORBIT": 11,
"LREAL": "Auto",
"LWAVE": False,
"LCHARG":False,
"NELM": 100,
"NSW": 900,
"SIGMA": 0.05,
"NPAR": 6
}

auto_kpoints = 25

Agora iremos criar a estrutura de pasta com todos os arquivos necessários para submeter as contas e todos os novos compostos:

In [ ]:
# Lista de elementos que serão usados para fazer as substituições químicas
earthalk_list = ['Be','Mg','Ca','Sr','Ba']
boron_list = ['B','Al','Ga','In','Tl']
halogen_list = ['F','Cl','Br','I']

In [ ]:
calculations_folder = 'ht/rx'
os.mkdir(calculations_folder)

for e1 in earthalk_list:
    for e2 in boron_list:
        for e3 in halogen_list:
            
            # Nova pasta para o novo material em disco
            output_folder=calculations_folder+'/'+e1+e2+e3+'5'
            os.mkdir(output_folder)
            
            temp_structure = proto_poscar.copy()          # Cópia da estrutura original em memória

            if not(e1=='Ca' and e2=='Al' and e3=='F'):
                for i, element in enumerate(proto_poscar):    # Para dada atomo na estrutura
                    element_str = str(element.species)        # Pega o atomo no sítio
                    element_str = element_str.replace('1','') # Remove o número de estequiometria (no caso o número é 1)

                    # Troca apenas elementos da mesma familia
                    if element_str in earthalk_list:
                        temp_structure.replace(i,e1)
                    elif element_str in boron_list:
                        temp_structure.replace(i,e2)
                    elif element_str in halogen_list:
                        temp_structure.replace(i,e3)

            # Aviso de que a eventualmente uma estrutura não manteve a simetria pretendida
            space_group = SpacegroupAnalyzer(temp_structure, symprec=0.1, angle_tolerance=1.0)
            if not(space_group.get_space_group_number()==proto_space_group.get_space_group_number()):
                print('Warning: Estrutura '+e1+e2+e3+'5 com grupo espacial diferente do protótipo!') 

            # Criar o novo POSCAR na pasta de destino
            new_poscar = vasp.inputs.Poscar(temp_structure)
            new_poscar.write_file(output_folder+'/POSCAR')
            
            # Criar o arquivo KPOINTS na pasta de destino
            kpoints = vasp.Kpoints().automatic(auto_kpoints)       # Método automatico com auto_kpoints (25) pontos
            kpoints.write_file(output_folder+'/KPOINTS')
            
            # Criar arquivo INCAR na pasta de destino
            incar = vasp.Incar(incar_rx)
            incar.write_file(output_folder+'/INCAR')
            
            # Criar arquivo POTCAR na pasta de destino
            
            
            # Criar arquivo JOB na pasta de destino
            